In [1]:
# Please note that DR(distribution rate) was previously referred to as ER(entanglement rate)
import numpy as np
import json

import tqdm as tqdm

from simplequantnetsim.protocols import MPC_protocol, MPG_protocol, SP_protocol
from simplequantnetsim.graph import network, set_edge_length, update_graph_params, set_p_edge
from simplequantnetsim.graph_manager import load_from_file
import networkx as nx

import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter

In [2]:
mkr = ["x", "+", "d", "o", "1", "2"] + ["x"] * 100
dashs = ["-.", "--", ":", "-"] + ["-"] * 100
cols = ["gray", "g", "b", "orange", "r", "k", "purple"] + ["k"] * 100
linewidth = 2.2
mks = 5.5
fontsize = 14

# DR against operation for the MPC, MPG, and SP protocols

In [ ]:
Mesh_topologies = "USnet.txt" # "ARPAnet_distancia_MTBF_MTTR.txt", "EON_distancia_MTBF_MTTR.txt", "Eurocore.txt", "NSFnet.txt". "UKnet.txt". "USnet.txt"
# center = (size - 1) / 2
G = load_from_file(Mesh_topologies) # G = network(size, size)
nodes = list(G.nodes)
protocols = [MPC_protocol, MPG_protocol, SP_protocol]

In [ ]:
# 计算接近中心性
closeness_centralities = nx.closeness_centrality(G)

# 将节点按接近中心性高低排序
sorted_closeness_centralities = sorted(closeness_centralities.items(), key=lambda item: item[1], reverse=True)

# 输出排序后的节点和其接近中心性
for node, centrality in sorted_closeness_centralities:
    print(f"Node {node}: {centrality}")

In [ ]:
p_op_range = np.linspace(1.0, 0.1, 100) # length_range = np.linspace(0, 30, 30) # probability of imperfect operations
ER = np.zeros((len(protocols), len(p_op_range)))
timesteps = 1000
reps = 200
Q_c = 1####################################################################################################
# length = 1
loss_dB = 0.2

update_graph_params(G, Qc=Q_c)

# users = [
#     '8',
#     '18',
#     '4',
#     '11',
#     '17',
# ]  # UK

# users = [
#     "8",
#     "15",
#     "19",
#     "1",
#     "2",
# ] # ARPA

# users = [
#     "14",
#     "0",
#     "7",
#     "1",
#     "18",
# ] # EON

# users = [
#     "1",
#     "0",
#     "5",
#     "2",
#     "3",
# ] # Euro

# users = [
#     "5",
#     "1",
#     "10",
#     "9",
#     "6",
# ] # NSF

users = [
    "22",
    "44",
    "42",
    "0",
    "43",
] # US

for i, p_op in enumerate(tqdm.tqdm(p_op_range)):  # tqdm_notebook
    set_p_edge(G, p_op, loss_dB=loss_dB) # set_edge_length(G, length=length, p_op=p_op, loss_dB=loss_dB)
    for j, function in enumerate(protocols):
        # if (
        #     (length > 160)
        #     # or (length > 160 and function == MPG_protocol)
        #     # or (length > 160 and function == SP_protocol)
        # ):
        #     ER[j, i] += 0
        #     break
        er, multipartite_gen_time, links_used = function(G, users, timesteps=timesteps, reps=reps)
        ER[j, i] += er

## Plot figure

In [ ]:
nom_list = [str(p).split(" ")[1] for p in protocols]
plt.figure(figsize=(8, 6), dpi=600)
plt.grid(linewidth=0.5)
for i in range(len(protocols)):
    y = plt.plot(
        p_op_range,
        ER[i],
        color=cols[i],
        marker="x",
        linestyle="None",
        markersize=3,
        #  alpha = 0.5,
        #  linewidth=linewidth,
        label=nom_list[i],
    )
plt.yscale("log")
plt.legend(fontsize=10)
plt.tick_params(labelsize=fontsize)

plt.xlabel("Perfect operations probability $p_{op}$", fontsize=fontsize)
plt.ylabel("DR ($\mathregular{GHZ}_5/\ \\mathregular{T_{slot}}$)", fontsize=fontsize)

ax = plt.gca()
ax.set_xlim([0, 1])
ax.set_ylim([0.0001, 1])

# plt.savefig("figures_HS/DR_v_operation_UK.png", dpi=600)

## Save data

In [ ]:
data_to_save = {
    "Mesh_topologies": Mesh_topologies,
    "Q_c": Q_c,
    "loss_dB": loss_dB,
    "datapoints": p_op_range.size,
    "timesteps": timesteps,
    "reps": reps,
    "users": users,
    "protocols": nom_list,
    "p_op_range": p_op_range.tolist(),
    "ER": ER.tolist(),
}

filename = "DR_p_op/US/Q1"#######################################################################################

with open("data_HS/" + filename + ".json", "w") as file:
    file.write(json.dumps(data_to_save))

## Load and plot data

In [ ]:
# file_to_open = "DR_p_op/NSF/Q1"##########################################################################
# with open("data_HS/" + file_to_open + ".json", "r") as file:
#     loaded_data = json.loads(file.read())

loaded_data = []
files_to_open = ["UK/Q1", "ARPA/Q1", "EON/Q1", "Euro/Q1", "NSF/Q1", "US/Q1"]
for i, file_name in enumerate(files_to_open):
    with open("data_HS/DR_p_op/" + file_name + ".json", "r") as file:
        loaded_data.append(json.loads(file.read()))

# reduce points
# loaded_data['length_range'][::2]

In [ ]:
# def length2p_ph(l):
#     return 10 ** -(loaded_data["loss_dB"] * l / 10)

# def p_ph2length(p):
#     return -(np.log10(p) * 10) / loaded_data["loss_dB"]

# length2p_e = lambda l: length2p_ph(l) * loaded_data["p_op"]
# p_e2length = lambda p: p_ph2length(p / loaded_data["p_op"])

# def p_op2p_e(op):
#     return op * p_ph

# def p_e2p_op(e):

In [ ]:
# "Mesh_topologies"

fontsize = 12
Mesh_topologies_list  = ["UKnet", "ARPAnet", "EON", "Eurocore", "NSFnet", "USnet"]
plt.figure(figsize=(10, 6), dpi=600)
plt.grid(linewidth=0.5)
for i in range(len(Mesh_topologies_list)):
    y = plt.plot(
        loaded_data[i]["p_op_range"][::2],
        loaded_data[i]["ER"][2][::2],
        color=cols[i],
        marker="x",
        linestyle="None",
        markersize=3,
        #  alpha = 0.5,
        #  linewidth=linewidth,
        label=Mesh_topologies_list[i],
    )# different Mesh networks


# fontsize = 12
# # grid_sizes = ["5x5", "9x9", "15x15", "25x25"]
# protocol_list = loaded_data["protocols"]
# plt.figure(figsize=(10, 6), dpi=600)
# plt.grid(linewidth=0.5)
# for i in range(len(protocol_list)):
#     y = plt.plot(
#         loaded_data["p_op_range"][::2],
#         loaded_data["ER"][i][::2],
#         color=cols[i],
#         marker="x",
#         linestyle="None",
#         markersize=3,
#         #  alpha = 0.5,
#         #  linewidth=linewidth,
#         label=protocol_list[i],
#     )# different protocol

# plot ER=p line
# plt.plot(
#     np.linspace(0, 160, 200),
#     [length2p_e(l) for l in np.linspace(0, 160, 200)],
#     color="black",
#     linestyle="--",
#     linewidth="1",
#     alpha=0.8,
#     label="$\mathregular{p_e}$",
# )

plt.yscale("log")
plt.legend(fontsize=10)
plt.tick_params(labelsize=fontsize)

plt.xlabel("Perfect operations probability $p_{op}$", fontsize=fontsize)
plt.ylabel("DR ($\mathregular{GHZ}_5/\ \\mathregular{T_{slot}}$)", fontsize=fontsize)

ax = plt.gca()
ax.set_xlim([0, 1])
ax.set_ylim([0.0001, 1])

# ax2 = ax.secondary_xaxis(-0.15, functions=(length2p_ph, p_ph2length))
# ax2.set_xlabel("Photon transmission probability $p_{ph}$", fontsize=fontsize)

# l_ticks = ax.get_xticks()
# p_ph_ticks = length2p_ph(l_ticks)
# ax2.set_xticks(p_ph_ticks)
# ax2.xaxis.set_major_formatter(FormatStrFormatter("%.2f"))
# ax2.xaxis.set_tick_params(labelsize=fontsize)

# ax3 = ax.secondary_xaxis(-0.3, functions=(length2p_e, p_e2length))
# ax3.set_xlabel("Entanglement link generation probability $p_e$", fontsize=fontsize)

# p_e_ticks = length2p_e(l_ticks)
# ax3.set_xticks(p_e_ticks)
# ax3.xaxis.set_major_formatter(FormatStrFormatter("%.2f"))
# ax3.xaxis.set_tick_params(labelsize=fontsize)

plt.savefig("figures_HS/DR_v_operation_SP.png", dpi=600, bbox_inches="tight")#############################

# Links used against operation

In [ ]:
Mesh_topologies = "UKnet.txt" # "ARPAnet_distancia_MTBF_MTTR.txt", "EON_distancia_MTBF_MTTR.txt", "Eurocore.txt", "NSFnet.txt". "UKnet.txt". "USnet.txt"
G = load_from_file(Mesh_topologies)
nodes = list(G.nodes)
protocols = [MPC_protocol, MPG_protocol, SP_protocol]

In [ ]:
p_op_range = np.linspace(0, 1, 200)
links_used = np.zeros((len(protocols), len(p_op_range)))
timesteps = 1000
reps = 200
Q_c = 100000###########################################################################################################
loss_dB = 0.2

update_graph_params(G, Qc=Q_c)

users = [
    '8',
    '18',
    '4',
    '11',
    '17',
]  # "source" for SP & MPG will be the central node

for i, p_op in enumerate(tqdm.tqdm(p_op_range)):  # tqdm_notebook
    set_p_edge(G, p_op, loss_dB=loss_dB)
    for j, function in enumerate(protocols):
        er, multipartite_gen_time, avg_links_used = function(G, users, timesteps=timesteps, reps=reps)
        links_used[j, i] += avg_links_used

## Save data

In [ ]:
data_to_save = {
    "Mesh_topologies": Mesh_topologies,
    "Q_c": Q_c,
    "loss_dB": loss_dB,
    "datapoints": p_op_range.size,
    "timesteps": timesteps,
    "reps": reps,
    "users": users,
    "protocols": nom_list,
    "p_op_range": p_op_range.tolist(),
    "links_used": links_used.tolist(),
}

filename = "links_p_op/UK/QX"################################################################################

with open("data_HS/" + filename + ".json", "w") as file:
    file.write(json.dumps(data_to_save))

## Load and plot data

In [ ]:
file_to_open = "links_p_op/UK/QX"###############################################################################
with open("data_HS/" + file_to_open + ".json", "r") as file:
    loaded_data = json.loads(file.read())

In [ ]:
# def length2p_ph(l):
#     return 10 ** -(loaded_data["loss_dB"] * l / 10)

# def p_ph2length(p):
#     return -(np.log10(p) * 10) / loaded_data["loss_dB"]

# length2p_e = lambda l: length2p_ph(l) * loaded_data["p_op"]
# p_e2length = lambda p: p_ph2length(p / loaded_data["p_op"])

In [ ]:
fontsize = 12
protocol_list = loaded_data["protocols"]
plt.figure(figsize=(10, 6), dpi=600)
plt.grid(linewidth=0.5)
for i in range(len(protocol_list)):
    y = plt.plot(
        loaded_data["p_op_range"],
        loaded_data["links_used"][i],
        color=cols[i],
        marker="x",
        linestyle="None",
        markersize=3,
        label=protocol_list[i],
    )

# plt.yscale("log")
plt.legend(fontsize=10)
plt.tick_params(labelsize=fontsize)

plt.xlabel("Perfect operations probability $p_{op}$", fontsize=fontsize)
plt.ylabel("Number of links used (/reps)", fontsize=fontsize)

ax = plt.gca()
ax.set_xlim([0, 1])
# ax.set_ylim([0.0001, 1])

# ax2 = ax.secondary_xaxis(-0.15, functions=(length2p_ph, p_ph2length))
# ax2.set_xlabel("Photon transmission probability $p_{ph}$", fontsize=fontsize)

# l_ticks = ax.get_xticks()
# p_ph_ticks = length2p_ph(l_ticks)
# ax2.set_xticks(p_ph_ticks)
# ax2.xaxis.set_major_formatter(FormatStrFormatter("%.2f"))
# ax2.xaxis.set_tick_params(labelsize=fontsize)

# ax3 = ax.secondary_xaxis(-0.3, functions=(length2p_e, p_e2length))
# ax3.set_xlabel("Entanglement link generation probability $p_e$", fontsize=fontsize)

# p_e_ticks = length2p_e(l_ticks)
# ax3.set_xticks(p_e_ticks)
# ax3.xaxis.set_major_formatter(FormatStrFormatter("%.2f"))
# ax3.xaxis.set_tick_params(labelsize=fontsize)

plt.savefig("figures_HS/links_v_operation_UK_QX.png", dpi=600, bbox_inches="tight")################################